In [103]:
import numpy as np
import torch
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
# import GPyOpt
# import GPy
import os
import matplotlib as mpl
import matplotlib.tri as tri
# import ternary
import pickle
import datetime
from collections import Counter
import matplotlib.ticker as ticker
from sklearn import preprocessing
# import pyDOE
import random
from scipy.stats import norm
import time
from sklearn.ensemble import RandomForestRegressor
import copy 

import warnings
warnings.filterwarnings("ignore")



In [104]:
from typing import Optional

from botorch.models.gpytorch import GPyTorchModel
from gpytorch.distributions import MultivariateNormal
from gpytorch.kernels import ScaleKernel, RBFKernel, MaternKernel, ConstantKernel

from gpytorch.likelihoods import GaussianLikelihood
from gpytorch.means import ConstantMean
from gpytorch.models import ExactGP
from torch import Tensor

from botorch.acquisition.monte_carlo import MCAcquisitionFunction
from botorch.models.model import Model
from botorch.sampling.base import MCSampler
from botorch.sampling.normal import SobolQMCNormalSampler
from botorch.utils import t_batch_mode_transform

from botorch.acquisition import qExpectedImprovement, ExpectedImprovement, LogExpectedImprovement, ProbabilityOfImprovement, UpperConfidenceBound

from botorch.fit import fit_gpytorch_mll
from botorch.models import SingleTaskGP
from botorch.test_functions import Hartmann
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.optim import optimize_acqf


# Load materials dataset

In [105]:
# go to directory where datasets reside
# load a dataset
# dataset names = ['Crossed barrel', 'Perovskite', 'AgNP', 'P3HT', 'AutoAM']
dataset_name = './datasets/P3HT'
raw_dataset = pd.read_csv(dataset_name + '_dataset.csv')
raw_dataset

,P3HT content (%),D1 content (%),D2 content (%),D6 content (%),D8 content (%),Conductivity (measured) (S/cm)
0,45.00,0.00,55.00,0.00,0.00,12.77
1,75.00,0.00,25.00,0.00,0.00,13.19
2,30.00,0.00,70.00,0.00,0.00,14.78
3,30.00,0.00,70.00,0.00,0.00,16.34
4,45.00,0.00,55.00,0.00,0.00,16.94
...,...,...,...,...,...,...
228,38.29,47.37,0.95,0.60,12.87,697.40
229,44.30,54.68,0.04,0.29,0.78,731.11
230,38.92,49.28,0.09,0.95,10.67,738.10
231,44.30,54.68,0.04,0.29,0.78,772.94


In [106]:
feature_name = list(raw_dataset.columns)[:-1]
feature_name

['P3HT content (%)',
 'D1 content (%)',
 'D2 content (%)',
 'D6 content (%)',
 'D8 content (%)']

In [107]:
objective_name = list(raw_dataset.columns)[-1]
objective_name

'Conductivity (measured) (S/cm)'

# Formulate optimization as global minimization

In [108]:
# for P3HT/CNT, Crossed barrel, AutoAM, their original goals were to maximize objective value.
# here, we add negative sign to all of its objective values here 
# because default BO in the framework below aims for global minimization
# only P3HT/CNT, Crossed barrel, AutoAM need this line; Perovskite and AgNP do not need this line.
ds = copy.deepcopy(raw_dataset) 
ds[objective_name] = -raw_dataset[objective_name].values
ds

,P3HT content (%),D1 content (%),D2 content (%),D6 content (%),D8 content (%),Conductivity (measured) (S/cm)
0,45.00,0.00,55.00,0.00,0.00,-12.77
1,75.00,0.00,25.00,0.00,0.00,-13.19
2,30.00,0.00,70.00,0.00,0.00,-14.78
3,30.00,0.00,70.00,0.00,0.00,-16.34
4,45.00,0.00,55.00,0.00,0.00,-16.94
...,...,...,...,...,...,...
228,38.29,47.37,0.95,0.60,12.87,-697.40
229,44.30,54.68,0.04,0.29,0.78,-731.11
230,38.92,49.28,0.09,0.95,10.67,-738.10
231,44.30,54.68,0.04,0.29,0.78,-772.94


# Process dataset for pool-based active learning

In [109]:
# for some datasets, each input feature x could have been evaluated more than once.
# to perform pool-based active learning, we need to group the data by unique input feature x value. 
# for each unique x in design space, we only keep the average of all evaluations there as its objective value
ds_grouped = ds.groupby(feature_name)[objective_name].agg(lambda x: x.unique().mean())
ds_grouped = (ds_grouped.to_frame()).reset_index()
ds_grouped.dropna(inplace=True)
ds_grouped

,P3HT content (%),D1 content (%),D2 content (%),D6 content (%),D8 content (%),Conductivity (measured) (S/cm)
0,15.00,0.00,0.00,85.00,0.00,-45.970
1,16.15,0.46,25.38,51.42,6.55,-16.905
2,16.88,1.25,24.96,49.58,7.32,-20.030
3,17.71,0.98,25.34,50.30,5.68,-14.780
4,18.67,0.27,47.09,24.03,9.99,-14.080
...,...,...,...,...,...,...
173,93.87,0.24,0.56,0.24,5.11,-9.020
174,94.58,0.03,1.15,1.38,2.91,-9.710
175,94.81,0.30,0.28,0.08,4.53,-6.440
176,95.00,0.00,0.00,0.00,5.00,-26.190


In [110]:
# these are the input feature x and objective value y used in framework
X_feature = ds_grouped[feature_name].values

y = np.array(ds_grouped[objective_name].values)

assert len(ds_grouped) == len(X_feature) == len(y)

# total number of data in set
N = len(ds_grouped)
print(N)

178


# Benchmarking Framework parameters

In [111]:
# here are some parameters of the framework, feel free to modify for your own purposes

# number of ensembles. in the paper n_ensemble = 50.
n_ensemble = 50
# number of initial experiments
n_initial = 2
# number of top candidates, currently using top 5% of total dataset size
n_top = int(math.ceil(len(y) * 0.05))
# the top candidates and their indicies
top_indices = list(ds_grouped.sort_values(objective_name).head(n_top).index)

# random seeds used to distinguish between different ensembles
# there are 300 of them, but only first n_ensemble are used
seed_list = [4295, 8508, 326, 3135, 1549, 2528, 1274, 6545, 5971, 6269, 2422, 4287, 9320, 4932, 951, 4304, 1745, 5956, 7620, 4545, 6003, 9885, 5548, 9477, 30, 8992, 7559, 5034, 9071, 6437, 3389, 9816, 8617, 3712, 3626, 1660, 3309, 2427, 9872, 938, 5156, 7409, 7672, 3411, 3559, 9966, 7331, 8273, 8484, 5127, 2260, 6054, 5205, 311, 6056, 9456, 928, 6424, 7438, 8701, 8634, 4002, 6634, 8102, 8503, 1540, 9254, 7972, 7737, 3410, 4052, 8640, 9659, 8093, 7076, 7268, 2046, 7492, 3103, 3034, 7874, 5438, 4297, 291, 5436, 9021, 3711, 7837, 9188, 2036, 8013, 6188, 3734, 187, 1438, 1061, 674, 777, 7231, 7096, 3360, 4278, 5817, 5514, 3442, 6805, 6750, 8548, 9751, 3526, 9969, 8979, 1526, 1551, 2058, 6325, 1237, 5917, 5821, 9946, 5049, 654, 7750, 5149, 3545, 9165, 2837, 5621, 6501, 595, 3181, 1747, 4405, 4480, 4282, 9262, 6219, 3960, 4999, 1495, 6007, 9642, 3902, 3133, 1085, 3278, 1104, 5939, 7153, 971, 8733, 3785, 9056, 2020, 7249, 5021, 3384, 8740, 4593, 7869, 9941, 8813, 3688, 8139, 6436, 3742, 5503, 1587, 4766, 9846, 9117, 7001, 4853, 9346, 4927, 8480, 5298, 4753, 1151, 9768, 5405, 6196, 5721, 3419, 8090, 8166, 7834, 1480, 1150, 9002, 1134, 2237, 3995, 2029, 5336, 7050, 6857, 8794, 1754, 1184, 3558, 658, 6804, 8750, 5088, 1136, 626, 8462, 5203, 3196, 979, 7419, 1162, 5451, 6492, 1562, 8145, 8937, 8764, 4174, 7639, 8902, 7003, 765, 1554, 6135, 1689, 9530, 1398, 2273, 7925, 5948, 1036, 868, 4617, 1203, 7680, 7, 93, 3128, 5694, 6979, 7136, 8084, 5770, 9301, 1599, 737, 7018, 3774, 9843, 2296, 2287, 9875, 2349, 2469, 8941, 4973, 3798, 54, 2938, 4665, 3942, 3951, 9400, 3094, 2248, 3376, 1926, 5180, 1773, 3681, 1808, 350, 6669, 826, 539, 5313, 6193, 5752, 9370, 2782, 8399, 4881, 3166, 4906, 5829, 4827, 29, 6899, 9012, 6986, 4175, 1035, 8320, 7802, 3777, 6340, 7798, 7705]

In [112]:
class SimpleCustomGP(ExactGP, GPyTorchModel):

    _num_outputs = 1  # to inform GPyTorchModel API

    def __init__(self, train_X, train_Y, train_Yvar: Optional[Tensor] = None):
        # NOTE: This ignores train_Yvar and uses inferred noise instead.
        # squeeze output dim before passing train_Y to ExactGP
        super().__init__(train_X, train_Y.squeeze(-1), GaussianLikelihood())
        self.mean_module = ConstantMean()
        self.covar_module = ScaleKernel(
            base_kernel=MaternKernel(ard_num_dims=train_X.shape[-1]),
        )
        self.to(train_X)  # make sure we're on the right device/dtype

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)

In [113]:
# model = SimpleCustomGP(
#     train_X=torch.from_numpy(X_feature).float(),
#     train_Y=torch.from_numpy(y).float(),
# )

# mll = ExactMarginalLogLikelihood(model.likelihood, model)

# # fit the model
# fit_gpytorch_mll(mll)

# # define the qEI acquisition module
# qEI = qExpectedImprovement(model=model, best_f=torch.max(model.train_targets))


# # define optimization function
# candiate_point, score = optimize_acqf(
#     acq_function=qEI,
#     bounds = torch.tensor([[0.0] * 5, [1.0] * 5]),
#     q=1,
#     num_restarts=10,
#     raw_samples=512,
#     options={"batch_limit": 5, "maxiter": 200},
# )

# candiate_point, score


# GP's surrogate models

In [114]:
class CustomGP(ExactGP, GPyTorchModel):
    _num_outputs = 1  # to inform GPyTorchModel API

    def __init__(self, train_X, train_Y, kernel="RBF", ard=False, train_Yvar=None):
        super().__init__(train_X, train_Y.squeeze(-1), GaussianLikelihood())
        self.mean_module = ConstantMean()

        # Select kernel based on argument
        if kernel == "RBF":
            base_kernel = RBFKernel(ard_num_dims=train_X.shape[-1] if ard else None)
        elif kernel == "Matern":
            base_kernel = MaternKernel(ard_num_dims=train_X.shape[-1] if ard else None)
        else:
            raise ValueError(f"Unsupported kernel type: {kernel}")

    
        self.covar_module = ScaleKernel(base_kernel + ConstantKernel()) # adding ConstantKernel to the base_kernel to model the noise level
        self.to(train_X.device)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)


In [115]:
model = CustomGP(
    train_X=torch.from_numpy(X_feature).float(),
    train_Y=torch.from_numpy(y).float(),
    kernel="Matern",
    ard=True,
)

In [116]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Pool-based active learning framework

In [117]:
def pool_learning(X, y, model, kernel, acqui_fn, top_indices, n_initial, seed):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    np.random.seed(seed)
    
    
    indices_to_learn = [i for i in range(len(X))] # indices that are still in the pool
    indices_learned= [] # indices that have been learned
    train_indices = random.sample(range(N), n_initial) # start with n_initial random points
    
    train_obj = torch.from_numpy(y[train_indices]).unsqueeze(-1).float() # objective values of these points
    train_x = torch.from_numpy(X[train_indices]).float() # input features of these points
    
    for i in train_indices:
        indices_to_learn.remove(i) # remove these points from the pool
    
    indices_learned.extend(train_indices) # add these points to learned points
    
    cnt = 0 #this counts the number of times the top indices are learned
    
    top_indices_cnt = []
    for index in indices_learned:
        if index in top_indices:
            cnt += 1
        top_indices_cnt.append(cnt)
                
    i = 0
    
    while len(indices_learned) < len(X):
        # initialize the model and fit it
        train_x = torch.from_numpy(scaler.fit_transform(train_x)).float()
        # train_y = torch.from_numpy(scaler.fit_transform(train_obj)).float()
        train_x = train_x.to(device)
        train_obj = train_obj.to(device)
        assert len(train_x) == len(train_obj)
        GP_model = model(train_X=train_x, train_Y=train_obj)
        mll = ExactMarginalLogLikelihood(GP_model.likelihood, GP_model)
        fit_gpytorch_mll(mll)
        
        # apply the LogExpectedImprovement acquisition function
        best_value = train_obj.min()
        acqui = acqui_fn(model=GP_model,beta = 2, maximize=False).to(device)
        
        # Apply acquisition function to find the best index to learn next
        candidate_features = torch.from_numpy(X[indices_to_learn]).float().to(device)

        acq_values = np.array([acqui(i.unsqueeze(0)).detach().cpu().numpy() for i in candidate_features])
        # print(acq_values)
        next_index = indices_to_learn[np.argmax(acq_values)] # find the index with the highest acquisition value
        
        
        # add the new point to the training set
        train_indices.append(next_index)
        indices_to_learn.remove(next_index)
        indices_learned.append(next_index)
        
        # update the training set
        train_x = torch.from_numpy(X[train_indices]).float()
        train_obj = torch.from_numpy(y[train_indices]).unsqueeze(-1).float()
        
        #update record of top indices
        if next_index in top_indices:
            cnt += 1
        
        top_indices_cnt.append(cnt)
        if cnt == len(top_indices):
            break
        
    return indices_learned, top_indices_cnt
    

In [118]:
from botorch.settings import debug
debug._set_state(True)


In [119]:
results_singletaskGP = []

for i in range(n_ensemble):
    print(f"Running ensemble {i+1} with RBF kernel")
    model = SingleTaskGP
    acqui_fn = UpperConfidenceBound
    indices, top_indices_cnt = pool_learning(
        X=X_feature,
        y=y,
        model=model,
        kernel="RBF",
        acqui_fn=acqui_fn,
        top_indices=top_indices,
        n_initial=n_initial,
        seed=seed_list[i]
    )
    results_singletaskGP.append(top_indices_cnt)
    

print(results_singletaskGP)


Running ensemble 1 with RBF kernel
Running ensemble 2 with RBF kernel
Running ensemble 3 with RBF kernel
Running ensemble 4 with RBF kernel
Running ensemble 5 with RBF kernel
Running ensemble 6 with RBF kernel
Running ensemble 7 with RBF kernel
Running ensemble 8 with RBF kernel
Running ensemble 9 with RBF kernel
Running ensemble 10 with RBF kernel
Running ensemble 11 with RBF kernel
Running ensemble 12 with RBF kernel
Running ensemble 13 with RBF kernel
Running ensemble 14 with RBF kernel
Running ensemble 15 with RBF kernel
Running ensemble 16 with RBF kernel
Running ensemble 17 with RBF kernel
Running ensemble 18 with RBF kernel
Running ensemble 19 with RBF kernel
Running ensemble 20 with RBF kernel
Running ensemble 21 with RBF kernel
Running ensemble 22 with RBF kernel
Running ensemble 23 with RBF kernel
Running ensemble 24 with RBF kernel
Running ensemble 25 with RBF kernel
Running ensemble 26 with RBF kernel
Running ensemble 27 with RBF kernel
Running ensemble 28 with RBF kernel
R

In [120]:
#save the result into an np array of lists 
saved = np.array(results_singletaskGP, dtype=object)
print(saved)
np.save('results_RBF_UCB.npy', saved)

[list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9])
 list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9])
 list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9])
 list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 